# RAG with PDFs: Part 2
* Improving the RAG Chain.
* First draft of the frontend with a React Template and Tailwind CSS.

## IMPORTANT: Installation with the exact packages we used
* When you download a full stack app you need to make sure that both backend and frontend use the original packages in order to avoid potential errors caused by installing more modern versions of these packages.
* Since we used pip to install the original backend packages and froze them using pip freeze, you will now use "pip install -r requirements.txt" to install them. Since we also used poetry, you will also use "poetry install".
* Since we used npx to install the original frontend packages, you will now use "npm ci" to install them.
#### Download the code
* Download the code from the github repository.
#### Backend installation
* Since we used both pyenv and poetry to build this project, you will have to use the following approach to install the backend.
* In the terminal, make sure you are in the root directory of the project (v1-163-part2). Pay attention: the root directory of the project and the backend directory have an identic name. Do not mistake them, be sure you are in the root directory of the project now.
* **Create a virtual environment and use pip install to make sure you install the exact same packages we used**:
    * pyenv virtualenv 3.11.4 your-virtual-environment-name
    * pyenv activate your-virtual-environment-name
    * pip install -r requirements.txt
* **Go to the backend directory, create a virtual environment and use poetry install to make sure you install the exact same packages we used**:
    * cd v1-163-part2
    * poetry install
#### Frontend installation
* Open a second terminal window, make sure you are in the root directory of the project (v1-163-part2). Pay attention: the root directory of the project and the backend directory have an identic name. Do not mistake them, be sure you are in the root directory of the project now.
* **Go to the frontend directory, and use npm ci to make sure you install the exact same packages we used**:
    * cd frontend
    * npm ci
#### Ready to go!
* You can now see the code of the app in Visual Studio Code.
* Relax and review the following steps. Remember, since you have pre-installed the modules you will not have to re-install them again.

## Alternative way to start the app
* `poetry run langchain serve`

## Let's start improving rag_chain.py
* In the final_chain:
    * Let's remove line 42 `|StrOutputParser()` so it will be easier to get into the sources in the frontend.
    * Let's use `RunnableParallel()` syntax instead of the dictionary syntax. It does the same, it is just more into the new LCEL style of LangChain.
        * We will need to add at the top:
        * `from langchain_core.runnables import RunnableParallel`
    * With the second line of the second `RunnableParallel()` we will make it easier to get the source of each response in the frontend.
* If we run our app again, we will see that instead of getting the answer as a String, we get a JSON object. This does not look very pretty now, but very soon we will see how this help us in our frontend.

#### Note: RunnableParallel()
`RunnableParallel` is a concept used in the context of parallel computation within a Python application, particularly with the `langchain` library. It represents a way to structure and execute multiple operations (or "runnables") in parallel, rather than sequentially. This is especially useful in scenarios where different pieces of data can be processed independently, thus potentially improving performance by utilizing concurrent execution.

**Key Points about `RunnableParallel`:**

- **Dictionary of Runnables**: It acts like a dictionary where each key is associated with a runnable entity. A runnable can be anything that performs an operation, such as a function or a more complex object designed to execute some code. In the context of `langchain`, it seems these runnables are specially designed objects capable of performing operations in a way that's integrated with the library's infrastructure for handling tasks.

- **Parallel Execution**: The primary feature of `RunnableParallel` is its ability to execute each of its contained runnables in parallel. This means that instead of running each task one after the other, it can run multiple tasks at the same time, assuming the hardware and Python environment support parallel execution (e.g., multiple CPU cores and properly handling asynchronous I/O or multi-threading/multiprocessing).

- **Unified Input and Output**: Each runnable within a `RunnableParallel` receives the same input, and the output is collected into a dictionary that mirrors the structure of the `RunnableParallel`. This means that you can feed a single set of data into the `RunnableParallel`, and it will dispatch this data to each of its runnables. Then, it aggregates the results from each runnable into a single dictionary where each key corresponds to the results of its associated runnable.

- **Chaining with Other Runnables**: The output of a `RunnableParallel` can be easily used as the input to another runnable, facilitating the creation of complex pipelines where some operations are parallelized while others might be sequential. This is particularly useful for data processing tasks where certain operations do not depend on the results of others and thus can be executed simultaneously to save time.

- **Simplification of Data Flow**: It also mentions that `RunnableParallel` can simplify matching the output of one runnable to the input format of another, highlighting its utility in data transformation and preparation tasks where inputs to subsequent processing steps need to be structured in a specific way.

**Example Usage:**

In the example provided in the LangChain documentation, `RunnableParallel` is used to execute two different chains in parallel, one generating a joke about a given topic (`joke_chain`) and the other generating a poem about the same topic (`poem_chain`). This illustrates how different tasks, potentially requiring significant processing power (like generating text with a machine learning model), can be executed simultaneously to efficiently produce a result that includes both a joke and a poem.

**Efficiency:**

The efficiency of parallel execution is further demonstrated through the `timeit` examples, showing that running both the `joke_chain` and `poem_chain` together in a `RunnableParallel` (`map_chain`) takes roughly the same amount of time as running either one individually, illustrating the time-saving benefits of parallel execution.

In summary, `RunnableParallel` is a powerful tool for structuring and executing operations in parallel, making it easier to design efficient data processing pipelines that leverage concurrent execution for tasks that are independent of one another.

* For more info, see [LangChain documentation](https://python.langchain.com/docs/expression_language/primitives/parallel/).

## Let's start building the frontend
* We will build a simple frontend with React.js and Tailwind.
* For the sake of simplicity, we will use just one React.js component. If we were working in a real production app, we would probably divide the content in several React components.
* You will need to have node, npm and npx installed in your computer.
    * To confirm they are installed, you can do this in terminal:
        * `node -v`
        * `npm -v`
        * `npx -v` 
    * For MacOS, you can install them easily with Homebrew via Terminal: `brew install node`. For Windows and other systems, you can check how to do this asking chatGPT4 or Google.
* Let's use a react template for the frontend. In Terminal:
    * `npx create-react-app frontend --template typescript`
* This will create a new folder called frontend
* Let's move into it:
    * `cd frontend`
* Now let's install some necessary modules there:
    * `npm install @microsoft/fetch-event-source` (see note below)
    * `npm install -D tailwindcss`  (see note below)
    * `npx tailwindcss init` (this initializes TailWind and creates the file tailwind.config.js)
    * Update the content in `frontend/src/index.css`: remove the default content and add the tailwind helpers (see note below).

#### Note: the @microsoft/fetch-event-source module
Imagine you have a website, and you want to get live updates from a server—like scores from a live sports game, stock market updates, or messages in a chat app (this is our case now). Normally, your website would have to keep asking the server, "Do you have any updates? Do you have any updates?" over and over. This is a bit like constantly refreshing a webpage to see if there's anything new, which isn't very efficient.

**Server-Sent Events (SSE)** is a technology that flips this around. Instead of your website constantly asking the server for updates, the server sends new data to your website whenever there's something new. It's like having a friend who texts you live updates from a game, so you don't have to keep asking them, "What's the score now?"

Now, to make this easy to do in your website's code, there's a package called `@microsoft/fetch-event-source` that you can add to your project. Here's what it does in simpler terms:

1. **It uses modern web technology** (the Fetch API) to make connecting to the server straightforward and efficient. Think of it as using a smart app to get live updates, rather than sending lots of texts back and forth.
   
2. **It handles the live stream of updates** from the server. Whenever the server sends a new piece of data, like a new score in the game, this package makes sure your website knows about it right away.
   
3. **It's designed to be easy to use**. Once you add it to your project, it takes care of the tricky parts of receiving these live updates, so you can focus on what to do with the information—like updating the score on your website in real-time.

To start using it, you install it in your project with a command that looks like this: `npm install @microsoft/fetch-event-source`. This is like downloading and installing an app that your website can use to receive these live updates.

In essence, `@microsoft/fetch-event-source` helps your website get live updates from a server in an efficient and easy way, so you can show those updates to your users in real-time, without having to constantly check for new information.

#### Note: Tailwinds CSS
Imagine you're building a LEGO house, and you have all these different bricks (colors, shapes, sizes) you can use. Now, organizing these bricks and figuring out the best combinations every time you build a new section of the house can be time-consuming and a bit overwhelming.

**Tailwind CSS** is like a set of pre-organized LEGO kits for building websites. Instead of worrying about all the individual bricks (CSS properties like colors, margins, paddings), Tailwind gives you a bunch of ready-to-use kits (utility classes). These kits let you quickly snap together a beautiful, responsive website without having to dive into the nitty-gritty of CSS details every time you want to style something.

When you run:

`npm install -D tailwindcss`

You're downloading Tailwind CSS into your project. The `-D` part means you're adding it as a "development dependency," which is a fancy way of saying, "This is a tool I need while I'm building my website, but I don't need it to actually run the website once it's built."

**What does Tailwind CSS do?**

1. **Saves Time**: It saves you a lot of time and effort. Instead of writing detailed CSS by hand for every element on your website, you use Tailwind's utility classes. For example, adding a class for making the text blue instead of writing CSS code for it.

2. **Keeps Things Consistent**: Because you're using these predefined kits, it helps keep the look of your website consistent. You don't have to remember the exact shade of blue you used for a button; you just use the same Tailwind class for it every time.

3. **Responsive Design**: Tailwind makes it easy to build websites that look great on phones, tablets, and computers without a lot of extra work. It has built-in utilities specifically for handling different screen sizes.

In summary, by running `npm install -D tailwindcss`, you're equipping your web development toolkit with Tailwind CSS, which helps you style your website more efficiently and consistently with its set of utility classes. It's like upgrading from a box of mixed LEGO bricks to a well-organized set of kits that make building and customizing your LEGO house (or website) a breeze.

#### Note: Tailwind helpers
These lines are part of a Tailwind CSS file, typically found at the beginning of a custom CSS file when you're working with Tailwind CSS in a project. Tailwind CSS is a utility-first CSS framework for creating custom designs directly in your markup. It provides a way to quickly style your HTML by using predefined classes that can control layout, typography, colors, and more. Let's break down what each directive does:

1. **`@tailwind base;`**: This directive injects Tailwind's base styles into your CSS. The base styles are a set of modern, sensible defaults for raw HTML elements like `<h1>`, `<p>`, `<button>`, etc. These base styles are meant to smooth out cross-browser inconsistencies and provide a clean slate for designing your components.

2. **`@tailwind components;`**: This directive injects Tailwind's component classes into your CSS. Components in Tailwind are predefined classes for UI elements that you might need in your application, such as buttons, forms, and navigation bars. These are crafted from Tailwind's utility classes but are named and structured in a way that represents common UI components.

3. **`@tailwind utilities;`**: This injects all of Tailwind's utility classes into your CSS. Utility classes are the core feature of Tailwind and cover a wide range of CSS properties. They enable you to style your HTML by applying classes directly in your markup. For example, classes like `text-center` for center-aligned text, `bg-blue-500` for a blue background, or `p-4` for padding. The utility classes are designed to be composable and customizable, allowing for a high degree of design flexibility.

When you include these directives in your CSS file and process it with Tailwind's build tools (like during a Webpack, Vite, or PostCSS build process), Tailwind scans your HTML and other template files for classes it recognizes. Then, it generates the corresponding CSS for those classes, along with the base styles and any custom components you've defined, ensuring that your final stylesheet includes only the styles you're actually using, which can significantly reduce the size of your CSS file.

This approach allows developers to work more efficiently by writing less custom CSS and instead leveraging the utility-first classes provided by Tailwind to build responsive, maintainable, and scalable designs directly in their HTML.

## Let's prepare the React frontend
In terminal, make sure you are in v1-163-part2/v1-163-part2/frontend and write:
* `npm start` to start the frontend. In [http://localhost:3000/](http://localhost:3000/) you can see that right now we only can see a default React template. Keep this open and you will see the following changes in real time.

Edit the file tailwind.config.js:
* `content: ["./src/**/*.{js,jsx,ts,tsx}"]`
* this tells tailwind to look in the src directory for javascript and typescript files (files ending in .js, .jsx, .ts and .tsx) in order to add tailwind styles.
  
Let's start now to customize the frontend/src/App.tsx file.

## App.tsx will be the main file of our React frontend
The `.tsx` file extension is used with files written in TypeScript that also contain JSX syntax. Let's break down what this means:

- **TypeScript**: This is a superset of JavaScript developed by Microsoft. It adds static types to JavaScript, which can help catch errors and bugs during development. TypeScript files typically have a `.ts` extension.

- **JSX**: JSX stands for JavaScript XML. It's a syntax extension for JavaScript, popularized by frameworks like React, that allows you to write HTML elements in JavaScript and place them in the DOM without using `createElement()` and `appendChild()` methods. It makes the code that generates HTML much easier to write and understand. JSX files in a React project typically have a `.jsx` extension.

When combining TypeScript with JSX, the file extension becomes `.tsx`. This tells the TypeScript compiler (and your development environment) that the file contains both TypeScript code and JSX syntax. This is commonly seen in projects built with React, where components are written using TypeScript for type safety and JSX for rendering the UI.

## Customizing App.tsx
* Here we will use Tailwind CSS. To learn how to use Tailwind, check:
    * [Tailwind Documention](https://tailwindcss.com/docs/installation).
    * [Great Cheatsheet for Tailwind](https://nerdcave.com/tailwind-cheat-sheet). 
* See the changes in App.tsx:
    * div
    * header
    * main
    * div for messages
    * textarea to enter questions
    * button
    * footer
* It is still very basic, but we have now the skeleton of the frontend of our application.